<a href="https://colab.research.google.com/github/RemeAjayi/ds-job-detective/blob/main/bert_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece


     |████████████████████████████████| 41 kB 135 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=eebd494d81a683f87ee42c55a17b9ad74f062f1e153b2f557c2e3ab032738356
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=afb1117ddc9bcefea2c972e0dae4393b0dc94ed15ad31e7e2c9e8bb75095f807
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=c833d8ce0d4665c52092d5c69a74bed38470463508b55287f3025f54aeadbe54
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [3]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.5.0
Hub version:  0.12.0


In [4]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)

In [5]:
MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

In [6]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [7]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [8]:
FullTokenizer=bert.bert_tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids


In [9]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [10]:
# Get the GPU device name. Checking if exists
device_name = tf.test.gpu_device_name()

# device name should appear as /device:GPU:0 if present and running
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
#     raise SystemError('GPU device not found')
    print('checked')

checked


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
train = pd.read_csv('/content/drive/MyDrive/np_train_skills_no_commas.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19769 entries, 0 to 19768
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  19769 non-null  int64  
 1   Phrase      19768 non-null  object 
 2   Target      13004 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 463.5+ KB


In [13]:
train.head()

,Unnamed: 0,Phrase,Target
0,0,significant scope impact,0.0
1,1,monitor compliance,0.0
2,2,self-service reporting solutions,0.0
3,3,experience,0.0
4,4,strong attention detail commitment,1.0


In [14]:
train = train.dropna()
train.isnull().sum()

Unnamed: 0    0
Phrase        0
Target        0
dtype: int64

In [15]:
# splitting data 1st iter 85/15 2nd iter 80/20 
# ensuring shuffling due to the sequentual nature of the training set
X = train.Phrase
y = train.Target

X_train, X_test, y_train, y_test = train_test_split(
                                                    X, y, 
                                                    test_size = .2,
                                                    random_state = 42,
                                                    shuffle = True
                                                    )

print('{:>5,} training samples'.format(len(X_train)))
print('{:>5,} validation samples'.format(len(X_test)))

10,403 training samples
2,601 validation samples


In [16]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [17]:
def build_model():
  x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x = tf.keras.layers.Dense(32, activation='relu')(x)
  x = tf.keras.layers.Dense(24, activation='tanh')(x)
  x = tf.keras.layers.Dense(12, activation='relu')(x)
  out = tf.keras.layers.Dense(1, activation='softplus')(x)
  #x = tf.keras.layers.Dropout(0.2)(x)
  #out = tf.keras.layers.Dense(1, activation="softplus", name="dense_output")(x)

  model = tf.keras.models.Model(
        inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
  
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [ ]:
inputs=create_input_array(X_train)
model = build_model()
with tf.device('/device:GPU:0'):
  model.fit(inputs,y_train,epochs=6,batch_size=60,validation_split=0.2,shuffle=True)

100%|██████████| 10403/10403 [00:01<00:00, 7666.98it/s]


Epoch 1/6
 59/139 [===========>..................] - ETA: 1:47:16 - loss: 0.5684 - accuracy: 0.6983

In [ ]:
model.summary()

In [ ]:
test_inputs=create_input_array(X_test)

print(model.predict(test_inputs))

In [ ]:
# found it was necessary to make a directory on colab while saving the model for easier retrieval
model.save('bert_skills_clf.h5', overwrite=True, include_optimizer=True)
# # predictions before we clear and reload model
pre_save_preds = model.predict(test_inputs) 
# # # Clear and load model
model = None
model = build_model()
model.load_weights('bert_skills_clf.h5')

# # predictions after we clear and reload model
post_save_preds = model.predict(test_inputs) 
# Are they the same?
all(pre_save_preds == post_save_preds)

In [ ]:
accr = model.evaluate(test_inputs, y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1])

In [ ]:
# Test on (assuming) unseen data
unseen = np.array([ 
                   'mongodb',
                   'statistics , mathematics , biostatistics',
                   'experience in python', 
                   'language e.g',
                   'this is fucking bullshit', 
                   'the definition', 
                   'hard-worker',
                   'san francisco bay area' ,
                   'many things',
                   'passion for data',
                   ])

unseen_inputs=create_input_array(unseen)

In [ ]:
unseen_truth = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1]
accr = model.evaluate(unseen_inputs, unseen_truth)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
unseen_predictions = model.predict(unseen_inputs)

In [ ]:
predictions = pd.DataFrame(dict(list(zip(unseen, unseen_predictions))))
predictions.T